# Resnet 5.
resnet 4_2 -> 50d로 모델만 확장

# Import

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.model_selection import KFold

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
import random
import numpy as np
import torch

# ============================================================
# 재현성 보장 (Reproducibility)
# ============================================================

SEED = 42

def set_seed(seed=SEED):
    """모든 난수 생성기의 시드를 고정하여 재현성 보장"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # multi-GPU
    
    # CUDA 결정론적 동작 활성화
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # PyTorch 결정론적 알고리즘 사용 (PyTorch 1.8+)
    torch.use_deterministic_algorithms(True, warn_only=True)
    
    print(f"✅ Random Seed 고정 완료: {seed}")
    print(f"   - Python random: {seed}")
    print(f"   - NumPy: {seed}")
    print(f"   - PyTorch: {seed}")
    print(f"   - CUDA: {seed}")
    print(f"   - cuDNN Deterministic: True")
    print(f"   - cuDNN Benchmark: False")
    print(f"   - PyTorch Deterministic Algorithms: True")

# Seed 적용
set_seed(SEED)

# Data Load

In [ ]:
# 하이퍼파라미터
img_size = 256
batch_size = 64
num_classes = 10
num_epochs = 50
# learning_rate = 0.0003 -> train section
num_workers = 0
version = "5"

# 데이터 경로
base_dir = r'c:\Users\USER\PycharmProjects\DeepLearning-Term-Proj'
driver_csv_path = os.path.join(base_dir, 'data', 'driver_imgs_list.csv')
train_dir = os.path.join(base_dir, 'data', 'imgs', 'train')
test_dir = os.path.join(base_dir, 'data', 'imgs', 'test')
name = f'resnet_{version}'

print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")
print(f"이미지 크기: {img_size}x{img_size}")
print(f"배치 크기: {batch_size}")

## offline prepr

In [ ]:
# import cv2
# import os
# from tqdm import tqdm

# def apply_clahe_to_folder(src_root, dst_root, clip=2.0, tile=8, is_test=False):
#     os.makedirs(dst_root, exist_ok=True)
    
#     if is_test:
#         process_images(src_root, dst_root, clip, tile, desc="Test Images")
#         return
    
#     for label in sorted(os.listdir(src_root)):
#         src_dir = os.path.join(src_root, label)
#         dst_dir = os.path.join(dst_root, label)
#         os.makedirs(dst_dir, exist_ok=True)
        
#         process_images(src_dir, dst_dir, clip, tile, desc=f"Class {label}")


# def process_images(src_dir, dst_dir, clip, tile, desc="Processing"):
#     """개별 이미지에 CLAHE 적용"""
#     img_files = [f for f in os.listdir(src_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
#     for img_name in tqdm(sorted(img_files), desc=desc):
#         src_path = os.path.join(src_dir, img_name)
#         dst_path = os.path.join(dst_dir, img_name)
        
#         if os.path.exists(dst_path):
#             continue
        
#         img = cv2.imread(src_path)
#         if img is None:
#             print(f"⚠️ 읽기 실패: {src_path}")
#             continue
        
#         img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
#         l, a, b = cv2.split(img_lab)
        
#         clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=(tile, tile))
#         l_clahe = clahe.apply(l)
        
#         img_clahe = cv2.merge([l_clahe, a, b])
#         img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_LAB2BGR)
        
#         cv2.imwrite(dst_path, img_clahe)

# apply_clahe_to_folder(
#     src_root=os.path.join(base_dir, 'data', 'imgs', 'train'),
#     dst_root=os.path.join(base_dir, 'data', 'clahe_imgs', 'train'),
#     clip=2.0,
#     tile=8
# )

# apply_clahe_to_folder(
#     src_root=os.path.join(base_dir, 'data', 'imgs', 'test'),
#     dst_root=os.path.join(base_dir, 'data', 'clahe_imgs', 'test'),
#     clip=2.0,
#     tile=8,
# 	is_test = True
# )

In [ ]:
clahe_train_dir = os.path.join(base_dir, 'data', 'clahe_imgs', 'train')
clahe_test_dir = os.path.join(base_dir, 'data', 'clahe_imgs', 'test')

In [ ]:
driver_df = pd.read_csv(driver_csv_path)

print(f"고유 운전자 수: {driver_df['subject'].nunique()}명")
print(f"운전자 목록: {sorted(driver_df['subject'].unique())}")

# driver_counts = driver_df['subject'].value_counts().sort_index()

# plt.figure(figsize=(12, 6))
# plt.bar(driver_counts.index.astype(str), driver_counts.values, color='C0', alpha=0.9)
# plt.xlabel('Driver')
# plt.ylabel('Image Count')
# plt.title('Images per Driver')
# plt.xticks(rotation=45, ha='right')
# plt.grid(axis='y', alpha=0.3)
# plt.tight_layout()
# plt.show()

# 5-Fold Cross Validation
fold별 운전자 목록을 미리 나누긴 하되 초기 실험 단계에서는 한 폴드(fold 2, 학습 데이터가 가장 많아서..)만 사용하고 이후 마무리 단계에서 전체 폴드 다 돌려서 일반화 성능 끌어올리는 방향으로 진행.

## split train data into 5 folds

In [ ]:
all_drivers = sorted(driver_df['subject'].unique())

n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

fold_splits = []
for fold_idx, (train_indices, val_indices) in enumerate(kfold.split(all_drivers)):
	train_drivers = [all_drivers[i] for i in train_indices]
	val_drivers = [all_drivers[i] for i in val_indices]

	fold_splits.append({
		'fold': fold_idx+1,
		'train_drivers': train_drivers,
		'val_drivers': val_drivers
	})

	print("Fold", fold_idx+1)
	print("train:", train_drivers, "val:", val_drivers)
	
	train_imgs = driver_df[driver_df['subject'].isin(train_drivers)]
	val_imgs = driver_df[driver_df['subject'].isin(val_drivers)]
	print(f"학습 이미지: {len(train_imgs)}개")
	print(f"검증 이미지: {len(val_imgs)}개")

## define DraiverDataset

In [ ]:
class DriverDataset(Dataset):
    """운전자 행동 데이터셋"""
    
    def __init__(self, data_dir, driver_df, driver_list, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.images = []
        self.labels = []
        
        if is_test:
            test_images_dir = data_dir
            for img_name in os.listdir(test_images_dir):
                self.images.append(os.path.join(test_images_dir, img_name))
        else: #is_train
            driver_subset = driver_df[driver_df['subject'].isin(driver_list)]

            for _, row in driver_subset.iterrows():
                class_name = row['classname']
                img_name = row['img']
                img_path = os.path.join(data_dir, class_name, img_name)

                self.images.append(img_path)
                class_idx = int(class_name[1:])
                self.labels.append(class_idx)
        print(f"{'테스트' if is_test else '운전자' + str(len(driver_list))+'명'}, 데이터 {len(self.images)}개 이미지")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, os.path.basename(img_path)
        else:
            label = self.labels[idx]
            return image, label

## define online transform
baseline 기본 성능을 확인하고자 resize, to tensor, normalize만 적용

In [ ]:
transform_train = transforms.Compose([
	transforms.Resize((img_size, img_size)),

	transforms.ToTensor(),
	transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

transform_eval = transforms.Compose([
	transforms.Resize((img_size, img_size)),
	transforms.ToTensor(),
	transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

## define model

### define custom resnet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
from torch.utils.data import DataLoader
import timm
import numpy as np
from sklearn.metrics import f1_score, log_loss
from tqdm import tqdm

EMBED_DIM = 512
DROPOUT = 0.4
DROP_PATH = 0.1
EARLY_STOP_PATIENCE = 10 

class ResNetDWithHead(nn.Module):
    def __init__(self, num_classes, embed_dim=512, dropout=0.4):
        super().__init__()
        self.backbone = timm.create_model(
            'resnet50d',
            pretrained=True,
            num_classes=0,
            global_pool='',
            drop_path_rate=DROP_PATH,
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        in_ch = self.backbone.num_features

        self.embed = nn.Sequential(
            nn.Conv2d(in_ch, in_ch, kernel_size=1, bias=False),
            nn.BatchNorm2d(in_ch),
            nn.ReLU(inplace=True),
        )
        self.proj = nn.Sequential(
            nn.Flatten(1),
            nn.Linear(in_ch, embed_dim),
            nn.BatchNorm1d(embed_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
        )
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward_features_map(self, x):
        return self.backbone.forward_features(x)

    def forward(self, x, labels=None):
        x = self.forward_features_map(x)
        x = self.embed(x)
        x = self.pool(x).flatten(1)
        x = self.proj(x)

        logits = self.classifier(x)
        return logits


## train

In [ ]:
def worker_init_fn(worker_id):
    """DataLoader worker의 재현성 보장"""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
def train_fold(fold_idx, train_drivers, val_drivers):
    print(f"==== Fold {fold_idx}/{n_folds} ====")

    # ===== 데이터셋 & 로더 =====
    g = torch.Generator()
    g.manual_seed(SEED)

    train_dataset = DriverDataset(
        clahe_train_dir, driver_df, train_drivers,
        transform=transform_train, is_test=False
    )
    val_dataset = DriverDataset(
        clahe_train_dir, driver_df, val_drivers,
        transform=transform_eval, is_test=False
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=True,
                              worker_init_fn=worker_init_fn, generator=g)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=True,
                            worker_init_fn=worker_init_fn, generator=g)

    print(f"학습 배치 수: {len(train_loader)}")
    print(f"검증 배치 수: {len(val_loader)}")

    # ===== 모델 =====
    model = ResNetDWithHead(
        num_classes=num_classes,
        embed_dim=EMBED_DIM,
        dropout=DROPOUT,
    ).to(device)

    # ---- layer1까지 동결 (stem~layer1 freeze) ----
    for n, p in model.backbone.named_parameters():
        if n.startswith('layer2') or n.startswith('layer3') or n.startswith('layer4'):
            p.requires_grad = True
        else:
            p.requires_grad = False
    # head는 학습
    for p in model.embed.parameters(): p.requires_grad = True
    for p in model.proj.parameters(): p.requires_grad = True
    for p in model.classifier.parameters(): p.requires_grad = True

    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print("\n" + "="*70)
    print("📊 모델 파라미터 통계")
    print("="*70)
    print(f"  전체 파라미터:      {total_params:>15,}")
    print(f"  학습 가능 파라미터:  {trainable_params:>15,} ({100*trainable_params/total_params:>6.2f}%)")
    print("="*70 + "\n")

    # ===== 손실/옵티마/스케줄러 =====
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=3e-4,
        weight_decay=5e-2
    )
    warmup_epochs = 5
    cosine_epochs = max(1, num_epochs - warmup_epochs)
    scheduler = SequentialLR(
        optimizer,
        schedulers=[
            LinearLR(optimizer, start_factor=0.01, end_factor=1.0, total_iters=warmup_epochs),
            CosineAnnealingLR(optimizer, T_max=cosine_epochs)
        ],
        milestones=[warmup_epochs]
    )

    # ===== 기록 =====
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [],
               'val_macro_f1': [], 'val_logloss': [], 'learning_rates': []}

    # ===== Early Stopping =====
    early_stop_patience = EARLY_STOP_PATIENCE
    patience_counter = 0
    best_metric = float('inf')
    best_model_path = f'models/{name}.pth'

    print(f"\n⏱️ Early Stopping Patience (val logloss 기준): {early_stop_patience} ep\n")
    print("="*70); print("🚀 학습 시작"); print("="*70)

    def eval_on_loader(model, loader, criterion):
        model.eval()
        total, correct, running_loss = 0, 0, 0.0
        all_probs, all_labels = [], []
        with torch.no_grad():
            pbar = tqdm(loader, desc='Validating', leave=False)
            for images, labels in pbar:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                logits = model(images)

                loss = criterion(logits, labels)
                running_loss += loss.item() * labels.size(0)
                preds = logits.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                probs = logits.softmax(dim=1).detach().cpu().numpy()
                all_probs.append(probs)
                all_labels.append(labels.detach().cpu().numpy())
                pbar.set_postfix({'loss': f'{loss.item():.4f}',
                                  'acc': f'{100.0*correct/max(1,total):.2f}%'})

        avg_loss = running_loss / max(1, total)
        acc = 100.0 * correct / max(1, total)
        all_probs = np.concatenate(all_probs, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        macro_f1 = f1_score(all_labels, np.argmax(all_probs, axis=1), average='macro')
        mlogloss = log_loss(all_labels, all_probs, labels=list(range(num_classes)))
        return avg_loss, acc, macro_f1, mlogloss

    # ===== 에폭 루프 =====
    for epoch in range(num_epochs):
        print(f'\n{"="*70}')
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'{"="*70}')
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)

        # ---- Train ----
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        pbar = tqdm(train_loader, desc='Training', leave=False)
        for images, labels in pbar:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            logits = model(images)

            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * labels.size(0)
            preds = logits.argmax(dim=1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

            pbar.set_postfix({'loss': f'{loss.item():.4f}',
                              'acc': f'{100.0*train_correct/max(1,train_total):.2f}%',
                              'lr': f'{current_lr:.6f}'})

        epoch_train_loss = train_loss / max(1, train_total)
        epoch_train_acc = 100.0 * train_correct / max(1, train_total)

        # ---- Validate ----
        val_loss, val_acc, val_macro_f1, val_logloss = eval_on_loader(model, val_loader, criterion)
        scheduler.step()

        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_macro_f1'].append(val_macro_f1)
        history['val_logloss'].append(val_logloss)

        print(f'\n📊 Epoch {epoch+1} 결과:')
        print(f'  Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}%')
        print(f'  Val   Loss: {val_loss:.4f} | Val Acc:   {val_acc:.2f}%')
        print(f'  Val Macro-F1: {val_macro_f1:.4f} | Val LogLoss: {val_logloss:.4f}')
        print(f'  LR: {current_lr:.6f}')

        # ---- Best / Early Stop ----
        if val_logloss < best_metric:
            best_metric = val_logloss
            patience_counter = 0
            torch.save({
                'fold': fold_idx,
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history,
                'val_loss': val_loss,
                'val_acc': val_acc,
                'val_macro_f1': val_macro_f1,
                'val_logloss': val_logloss,
                'model_name': name,
                'freeze_mode': 'freeze_to_layer1',
                'drop_path_rate': DROP_PATH,
                'trainable_params': trainable_params,
                'total_params': total_params,
                'seed': SEED
            }, best_model_path)
            print(f'  ✅ 최고 성능(val_logloss) 모델 저장! (val_logloss: {val_logloss:.4f})')
        else:
            patience_counter += 1
            print(f'  ⏳ Early Stopping 카운터: {patience_counter}/{EARLY_STOP_PATIENCE}')
            if patience_counter >= EARLY_STOP_PATIENCE:
                print(f'\n{"="*70}')
                print(f'🛑 Early Stopping 발동! (Epoch {epoch+1}) — val_logloss 개선 없음')
                print(f'   최고 val_logloss: {best_metric:.4f}')
                print(f'   모델 저장 경로: {best_model_path}')
                print(f'{"="*70}')
                break

    final_epoch = epoch + 1
    print("\n" + "="*70)
    print(f"✅ Fold {fold_idx} 학습 완료! (EXP-1+)")
    print("="*70)
    print(f"  총 학습 에폭: {final_epoch}/{num_epochs}")
    print(f"  최고 Macro-F1: {max(history['val_macro_f1']):.4f}")
    print(f"  최저 Val Loss: {min(history['val_loss']):.4f}")
    print(f"  최저 LogLoss:  {min(history['val_logloss']):.4f}")
    print(f"  최고 Val Acc:  {max(history['val_acc']):.2f}%")
    print(f"  모델 저장: {best_model_path}")
    print(f"  학습 파라미터: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.2f}%)")
    print("="*70)

    return {
        'fold': fold_idx,
        'history': history,
        'best_macro_f1': max(history['val_macro_f1']),
        'best_val_loss': min(history['val_loss']),
        'best_val_logloss': min(history['val_logloss']),
        'best_val_acc': max(history['val_acc']),
        'model_path': best_model_path,
        'stopped_epoch': final_epoch,
        'model_name': name,
        'freeze_mode': 'freeze_to_layer1',
        'trainable_params': trainable_params,
        'total_params': total_params,
        'seed':SEED
    }


In [ ]:
# ========== Fold 2 학습 (Inception-C부터 Fine-tuning) ==========

all_fold_results = []

fold_info = fold_splits[1]  # Fold 2
fold_idx = fold_info['fold']
train_drivers = fold_info['train_drivers']
val_drivers = fold_info['val_drivers']


print(f"==== {name} ====")


fold_result = train_fold(
    fold_idx, 
    train_drivers, 
    val_drivers,
)

all_fold_results.append(fold_result)

# 메모리 정리
torch.cuda.empty_cache()

# 결과 출력
print(f"\n{'='*70}")
print("📊 최종 결과")
print(f"{'='*70}")
print(f"  Fold: {fold_result['fold']}")
print(f"  최저 Val Loss: {fold_result['best_val_loss']:.4f}")
print(f"  해당 Val Acc: {fold_result['best_val_acc']:.2f}%")
print(f"  학습 완료 에폭: {fold_result['stopped_epoch']}")
print(f"  Freeze 모드: {fold_result['freeze_mode']}")
print(f"  학습 파라미터: {fold_result['trainable_params']:,} / {fold_result['total_params']:,}")
print(f"{'='*70}")

In [ ]:
# ========== 학습 곡선 시각화 (4개 그래프) ==========

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for result in all_fold_results:
    fold_idx = result['fold']
    history = result['history']
    stopped_epoch = result['stopped_epoch']
    best_val_loss = result['best_val_loss']
    
    # 1. Loss 그래프
    ax1 = axes[0, 0]
    epochs = range(1, len(history['train_loss']) + 1)
    ax1.plot(epochs, history['train_loss'], label='Train Loss', 
             marker='o', linewidth=2, alpha=0.8, color='#1f77b4')
    ax1.plot(epochs, history['val_loss'], label='Val Loss', 
             marker='s', linewidth=2, alpha=0.8, color='#ff7f0e')
    
    # 최저 Val Loss 지점 표시
    best_epoch = np.argmin(history['val_loss']) + 1
    ax1.scatter(best_epoch, best_val_loss, color='red', s=250, zorder=5, 
                marker='*', edgecolors='black', linewidths=2,
                label=f'Best (Epoch {best_epoch})')
    
    # Early Stopping 지점 표시
    if stopped_epoch < num_epochs:
        ax1.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5, label=f'Early Stop (E{stopped_epoch})')
    
    ax1.set_title(f'Fold {fold_idx} - Loss (Multiclass Log Loss)', 
                 fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.legend(loc='best', fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # 2. Accuracy 그래프
    ax2 = axes[0, 1]
    ax2.plot(epochs, history['train_acc'], label='Train Acc', 
             marker='o', linewidth=2, alpha=0.8, color='#2ca02c')
    ax2.plot(epochs, history['val_acc'], label='Val Acc', 
             marker='s', linewidth=2, alpha=0.8, color='#d62728')
    
    # 최고 Val Acc 지점 표시
    best_acc_epoch = np.argmax(history['val_acc']) + 1
    best_val_acc = max(history['val_acc'])
    ax2.scatter(best_acc_epoch, best_val_acc, color='green', s=250, zorder=5,
                marker='*', edgecolors='black', linewidths=2,
                label=f'Best (Epoch {best_acc_epoch})')
    
    if stopped_epoch < num_epochs:
        ax2.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5, label=f'Early Stop (E{stopped_epoch})')
    
    ax2.set_title(f'Fold {fold_idx} - Accuracy', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy (%)', fontsize=12)
    ax2.legend(loc='best', fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    # 3. Learning Rate 그래프
    ax3 = axes[1, 0]
    ax3.plot(epochs, history['learning_rates'], marker='o', linewidth=2, 
             color='purple', alpha=0.8, label='Learning Rate')
    ax3.set_title(f'Fold {fold_idx} - Learning Rate Schedule', 
                 fontsize=14, fontweight='bold')
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('Learning Rate', fontsize=12)
    ax3.set_yscale('log')
    ax3.legend(loc='best', fontsize=10)
    ax3.grid(True, alpha=0.3, which='both')
    
    if stopped_epoch < num_epochs:
        ax3.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5)
    
    # 4. Train vs Val 비교 (Loss & Acc)
    ax4 = axes[1, 1]
    
    # Loss 차이
    loss_diff = np.array(history['train_loss']) - np.array(history['val_loss'])
    ax4_twin = ax4.twinx()
    
    ax4.plot(epochs, loss_diff, marker='o', linewidth=2, 
            color='#e377c2', alpha=0.7, label='Loss Diff (Train - Val)')
    ax4.axhline(0, color='gray', linestyle='--', linewidth=1)
    ax4.set_xlabel('Epoch', fontsize=12)
    ax4.set_ylabel('Loss Difference', fontsize=12, color='#e377c2')
    ax4.tick_params(axis='y', labelcolor='#e377c2')
    
    # Accuracy 차이
    acc_diff = np.array(history['train_acc']) - np.array(history['val_acc'])
    ax4_twin.plot(epochs, acc_diff, marker='s', linewidth=2,
                 color='#bcbd22', alpha=0.7, label='Acc Diff (Train - Val)')
    ax4_twin.set_ylabel('Accuracy Difference (%)', fontsize=12, color='#bcbd22')
    ax4_twin.tick_params(axis='y', labelcolor='#bcbd22')
    
    ax4.set_title(f'Fold {fold_idx} - Overfitting 모니터링', 
                 fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    # Legend 통합
    lines1, labels1 = ax4.get_legend_handles_labels()
    lines2, labels2 = ax4_twin.get_legend_handles_labels()
    ax4.legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=9)
    
    if stopped_epoch < num_epochs:
        ax4.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5)

plt.tight_layout()
os.makedirs(f'./plots/{name}', exist_ok=True)
plt.savefig(f'./plots/{name}/{name}_losscurve_detailed.png', 
           dpi=300, bbox_inches='tight')
plt.show()

# ========== 통계 출력 ==========
print("\n" + "="*70)
print(f"📈 {name} 학습 통계 상세")
print("="*70)
print(f"초기 Train Loss: {history['train_loss'][0]:.4f}")
print(f"최종 Train Loss: {history['train_loss'][-1]:.4f}")
print(f"최저 Train Loss: {min(history['train_loss']):.4f} (Epoch {np.argmin(history['train_loss'])+1})")
print(f"\n초기 Val Loss: {history['val_loss'][0]:.4f}")
print(f"최종 Val Loss: {history['val_loss'][-1]:.4f}")
print(f"최저 Val Loss: {min(history['val_loss']):.4f} (Epoch {np.argmin(history['val_loss'])+1})")
print(f"\n초기 Train Acc: {history['train_acc'][0]:.2f}%")
print(f"최종 Train Acc: {history['train_acc'][-1]:.2f}%")
print(f"최고 Train Acc: {max(history['train_acc']):.2f}% (Epoch {np.argmax(history['train_acc'])+1})")
print(f"\n초기 Val Acc: {history['val_acc'][0]:.2f}%")
print(f"최종 Val Acc: {history['val_acc'][-1]:.2f}%")
print(f"최고 Val Acc: {max(history['val_acc']):.2f}% (Epoch {np.argmax(history['val_acc'])+1})")
print(f"\n초기 LR: {history['learning_rates'][0]:.6f}")
print(f"최종 LR: {history['learning_rates'][-1]:.6f}")
print(f"LR 변경 횟수: {len(set(history['learning_rates'])) - 1}회")
print("="*70)

# ========== Overfitting 분석 ==========
final_loss_gap = history['train_loss'][-1] - history['val_loss'][-1]
final_acc_gap = history['train_acc'][-1] - history['val_acc'][-1]

print("\n" + "="*70)
print("🔍 Overfitting 분석")
print("="*70)
print(f"최종 Loss 차이 (Train - Val): {final_loss_gap:+.4f}")
print(f"최종 Acc 차이 (Train - Val): {final_acc_gap:+.2f}%")

if final_acc_gap > 10:
    print("⚠️ 경고: 심각한 Overfitting 감지! (Acc 차이 > 10%)")
elif final_acc_gap > 5:
    print("⚠️ 주의: 약간의 Overfitting 감지 (Acc 차이 > 5%)")
else:
    print("✅ 양호: Overfitting이 잘 제어되고 있습니다.")

print("="*70)

# Submission

In [ ]:
# ========== Submission 파일 생성 (학습된 모델 사용) ==========

print("\n" + "=" * 70)
print("🔮 테스트 데이터 예측 시작")
print("=" * 70)

# 1. 테스트 데이터셋 생성
test_dataset = DriverDataset(
    clahe_test_dir, driver_df, [], 
    transform=transform_eval, is_test=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f"테스트 샘플: {len(test_dataset)}개")

# 2. 모델 생성
model = ResNetDWithHead(
    num_classes=num_classes,
    embed_dim=EMBED_DIM,
    dropout=DROPOUT,
).to(device)

# 3. 저장된 가중치 로드
best_model_path = f'models/{name}.pth'
print(f"\n📁 모델 로드 중: {best_model_path}...")

try:
    checkpoint = torch.load(best_model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # 로드된 모델 정보 출력
    print(f"✅ 모델 로드 완료!")
    print(f"   - Fold: {checkpoint.get('fold', 'N/A')}")
    print(f"   - Epoch: {checkpoint.get('epoch', 'N/A')+1}")
    print(f"   - Val Loss: {checkpoint.get('val_loss', 'N/A'):.4f}")
    print(f"   - Val logLoss: {checkpoint.get('val_logloss', 'N/A'):.4f}")
    print(f"   - Val Acc: {checkpoint.get('val_acc', 'N/A'):.2f}%")
    print(f"   - Val Macro-F1: {checkpoint.get('val_macro_f1', 'N/A'):.4f}")
    print(f"   - Val LogLoss: {checkpoint.get('val_logloss', 'N/A'):.4f}")
    
except Exception as e:
    print(f"🚨 모델 로드 실패: {e}")
    raise

model.eval()

# 4. 예측 실행
predictions = []
img_names = []

print("\n🔮 예측 진행 중...")
with torch.no_grad():
    for images, filenames in tqdm(test_loader, desc='예측'):
        images = images.to(device, non_blocking=True)
        outputs = model(images)
        
        # 5. 확률 (Softmax) 계산
        probs = torch.softmax(outputs, dim=1)
        predictions.append(probs.cpu().numpy())
        img_names.extend(filenames)

final_predictions = np.vstack(predictions)
print(f"\n✅ 예측 완료: {final_predictions.shape}")

# 6. Submission 파일 생성
class_cols = [f'c{i}' for i in range(num_classes)]

submission_data = {'img': img_names}
for i, col in enumerate(class_cols):
    submission_data[col] = final_predictions[:, i]

submission = pd.DataFrame(submission_data)

# 7. 파일 저장
os.makedirs('./submissions', exist_ok=True)
submission.to_csv(f"./submissions/{name}_submission.csv", index=False)

print("\n" + "=" * 70)
print(f"✅ Submission 파일 생성 완료!")
print("=" * 70)
print(f"📁 저장 경로: ./submissions/{name}_submission.csv")
print(f"📊 총 예측 이미지: {len(submission)}개")
print(f"📋 클래스별 평균 확률:")
for col in class_cols:
    print(f"   {col}: {submission[col].mean():.4f}")
print("=" * 70)

print("\n📋 Submission 샘플 (처음 5개):")
print(submission.head())

print("\n📋 Submission 샘플 (마지막 5개):")
print(submission.tail())

# 8. 확률 분포 검증 (합이 1인지 확인)
prob_sums = submission[class_cols].sum(axis=1)
print(f"\n✅ 확률 합 검증: min={prob_sums.min():.6f}, max={prob_sums.max():.6f}, mean={prob_sums.mean():.6f}")
if not np.allclose(prob_sums, 1.0, atol=1e-5):
    print("⚠️ 경고: 일부 샘플의 확률 합이 1이 아닙니다!")
else:
    print("✅ 모든 샘플의 확률 합이 1입니다.")

# Analysis

In [ ]:
# 셀 추가: Confusion Matrix 분석
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def analyze_predictions(model, val_loader, device):
    """검증 데이터로 상세 분석"""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='예측 중'):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images, labels)
            probs = torch.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
    
    all_probs = np.vstack(all_probs)
    return np.array(all_labels), np.array(all_preds), all_probs

# 모델 로드
model = ResNetDWithHead(
    num_classes=num_classes,
    embed_dim=EMBED_DIM,
    dropout=DROPOUT,
).to(device)
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)


# 예측
fold_info = fold_splits[1] 

fold_idx = fold_info['fold']
train_drivers = fold_info['train_drivers']
val_drivers = fold_info['val_drivers']

val_dataset = DriverDataset(
    train_dir, driver_df, val_drivers,
    transform=transform_eval, is_test=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
)
y_true, y_pred, y_probs = analyze_predictions(model, val_loader, device)

# 1. Confusion Matrix
class_names = [f'c{i}' for i in range(10)]
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(f'./plots/{name}/confusion_matrix_{name}.png', dpi=300)
plt.show()

# 2. Normalized Confusion Matrix (비율)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='YlOrRd',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Normalized Confusion Matrix (%)', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
os.makedirs(f'plots/{name}', exist_ok=True)
plt.savefig(f'plots/{name}/confusion_matrix_normalized_{name}.png', dpi=300)
plt.show()

# 3. Classification Report
print("\n" + "="*70)
print("📊 Classification Report")
print("="*70)
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

In [ ]:
# 클래스별 성능 분석
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

if 'y_true' not in globals() or 'y_pred' not in globals() or 'y_probs' not in globals():
    print("y_true/y_pred/y_probs가 없음 — analyze_predictions 실행 중...")
    y_true, y_pred, y_probs = analyze_predictions(model, val_loader, device)
else:
    print("y_true/y_pred/y_probs 이미 존재, 재계산 생략")

precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, average=None
)

# 데이터프레임으로 정리
class_performance = pd.DataFrame({
    'Class': class_names,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support,
    'Accuracy': [accuracy_score(y_true[y_true==i], y_pred[y_true==i]) 
                 if np.sum(y_true==i) > 0 else 0 for i in range(10)]
})

print("\n" + "="*70)
print("📈 클래스별 성능")
print("="*70)
print(class_performance.to_string(index=False))

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Precision
axes[0, 0].bar(class_names, precision, color='skyblue', alpha=0.8)
axes[0, 0].set_title('Precision by Class', fontweight='bold')
axes[0, 0].set_ylabel('Precision')
axes[0, 0].set_ylim([0, 1.1])
axes[0, 0].grid(axis='y', alpha=0.3)

# Recall
axes[0, 1].bar(class_names, recall, color='lightcoral', alpha=0.8)
axes[0, 1].set_title('Recall by Class', fontweight='bold')
axes[0, 1].set_ylabel('Recall')
axes[0, 1].set_ylim([0, 1.1])
axes[0, 1].grid(axis='y', alpha=0.3)

# F1-Score
axes[1, 0].bar(class_names, f1, color='lightgreen', alpha=0.8)
axes[1, 0].set_title('F1-Score by Class', fontweight='bold')
axes[1, 0].set_ylabel('F1-Score')
axes[1, 0].set_ylim([0, 1.1])
axes[1, 0].grid(axis='y', alpha=0.3)

# Support
axes[1, 1].bar(class_names, support, color='plum', alpha=0.8)
axes[1, 1].set_title('Support (Sample Count) by Class', fontweight='bold')
axes[1, 1].set_ylabel('Count')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
os.makedirs(f'plots/{name}', exist_ok=True)
plt.savefig(f'plots/{name}/class_performance_{name}.png', dpi=300)
plt.show()

In [ ]:
# ROC Curve 분석
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# One-hot encoding
y_true_bin = label_binarize(y_true, classes=range(10))

# 각 클래스별 ROC Curve
plt.figure(figsize=(14, 10))

for i in range(10):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{class_names[i]} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves (Multi-class)', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(f'plots/{name}/roc_curves_{name}.png', dpi=300)
plt.show()

In [ ]:
# Grad-CAM 구현
import torch.nn.functional as F
from torchvision.transforms import ToPILImage

class GradCAM:
    """Grad-CAM (ResNet 호환)"""
    def __init__(self, model, target_module=None):
        self.model = model
        self.target_module = target_module
        self.gradients = None
        self.activations = None

        # target_module이 None이면 모델에서 마지막 Conv2d를 찾아 사용
        if self.target_module is None:
            self.target_module, _ = get_last_conv_layer_resnet(self.model)
            if self.target_module is None:
                raise RuntimeError("마지막 Conv 레이어를 찾을 수 없습니다.")

        # 훅 등록 (forward / backward)
        self.target_module.register_forward_hook(self._save_activation)
        # 최신 PyTorch에서는 register_full_backward_hook 권장, 없으면 register_backward_hook 사용
        if hasattr(self.target_module, "register_full_backward_hook"):
            self.target_module.register_full_backward_hook(self._save_gradient)
        else:
            # 구버전 호환
            self.target_module.register_backward_hook(self._save_gradient)

    def _save_activation(self, module, input, output):
        self.activations = output.detach()

    def _save_gradient(self, module, grad_input, grad_output):
        # grad_output는 tuple
        self.gradients = grad_output[0].detach()

    def generate_cam(self, input_tensor, target_class=None):
        """
        input_tensor: [1, C, H, W] (이미 정규화된 텐서)`
        target_class: int or None (None이면 argmax 사용)
        returns: cam (H, W) float numpy [0..1], logits tensor
        """
        self.model.eval()
        self.gradients = None
        self.activations = None
        logits = self.model(input_tensor)  # 일반 forward

        if target_class is None:
            target_class = int(logits.argmax(dim=1)[0])

        # backward on the score of target_class
        self.model.zero_grad()
        score = logits[0, target_class]
        score.backward(retain_graph=True)

        if self.gradients is None or self.activations is None:
            raise RuntimeError("Gradients or activations not recorded. Check hooks.")

        # gradients: [1, C, H, W] -> use [C, H, W]
        gradients = self.gradients[0].cpu()
        activations = self.activations[0].cpu()

        # global average pooling of gradients -> weights [C]
        weights = gradients.mean(dim=(1, 2))  # [C]

        # weighted sum of activations
        cam = (weights[:, None, None] * activations).sum(dim=0)  # [H, W]
        cam = F.relu(cam)
        cam = cam - cam.min()
        if cam.max() > 0:
            cam = cam / cam.max()
        cam_np = cam.numpy().astype(np.float32)

        return cam_np, logits

def get_last_conv_layer_resnet(model):
    """
    ResNet 계열 모델에서 마지막 nn.Conv2d 모듈을 찾아 반환
    반환: (module, name) 또는 (None, None)
    """
    for name, module in reversed(list(model.named_modules())):
        if isinstance(module, torch.nn.Conv2d):
            return module, name
    return None, None

def visualize_gradcam(model, image, true_label, pred_label, device):
    """Grad-CAM 시각화 (ResNet)"""
    # target layer 찾기/설정
    target_module, target_name = get_last_conv_layer_resnet(model)
    if target_module is None:
        print("⚠️ 마지막 Conv 레이어를 찾을 수 없습니다.")
        return

    gradcam = GradCAM(model, target_module)

    # 이미지는 [C, H, W] (정규화된 텐서)
    input_tensor = image.unsqueeze(0).to(device)
    cam, output = gradcam.generate_cam(input_tensor, target_class=pred_label)

    # 원본 이미지 복원 (정규화 역변환)
    img_np = image.cpu().numpy().transpose(1, 2, 0)
    img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img_np = np.clip(img_np, 0, 1)

    # cam은 [H, W], float [0..1] -> uint8로 변환 후 PIL 리사이즈
    H, W = img_np.shape[:2]
    cam_uint8 = np.uint8(255 * cam)
    cam_pil = Image.fromarray(cam_uint8).resize((W, H), Image.BILINEAR)
    cam_resized = np.array(cam_pil) / 255.0

    # 시각화
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(img_np)
    axes[0].set_title(f'Original\nTrue: c{true_label}', fontsize=12)
    axes[0].axis('off')

    axes[1].imshow(img_np)
    axes[1].imshow(cam_resized, cmap='jet', alpha=0.5)
    axes[1].set_title(f'Grad-CAM\nPred: c{pred_label}', fontsize=12)
    axes[1].axis('off')

    axes[2].imshow(cam_resized, cmap='jet')
    axes[2].set_title('Heatmap', fontsize=12)
    axes[2].axis('off')

    plt.tight_layout()
    return fig

# 샘플 이미지에 Grad-CAM 적용
print("\n" + "="*70)
print("🔥 Grad-CAM 시각화")
print("="*70)

# 모델 로드 (ResNetDWithHead 래퍼 사용)
model = ResNetDWithHead(
    num_classes=num_classes,
    embed_dim=EMBED_DIM,
    dropout=DROPOUT,
).to(device)

checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

# 검증 데이터에서 샘플 추출 (정답/오답 각각)
correct_samples = []
incorrect_samples = []

for i, (images, labels) in enumerate(tqdm(val_loader)):
    images = images.to(device)
    labels = labels.to(device)
    
    with torch.no_grad():
        outputs = model(images, labels)
        _, preds = torch.max(outputs, 1)
    
    for j in range(len(images)):
        if preds[j] == labels[j]:
            correct_samples.append((images[j], labels[j].item(), preds[j].item()))
        elif preds[j] != labels[j]:
            incorrect_samples.append((images[j], labels[j].item(), preds[j].item()))

In [ ]:
print("정답 예측 샘플 (각 클래스 별 5개)")
corr_class_samples = {i: [] for i in range(10)}
max_samples = 5
os.makedirs(f'plots/{name}/gradcam_correct', exist_ok=True)

for img, true_label, pred_label in correct_samples:
	curr = corr_class_samples[true_label]
	if len(curr) < max_samples:
		curr.append((img, true_label, pred_label))

for class_idx in range(10):
	if corr_class_samples[class_idx]:
		print(f"{class_idx} grad cam")
		for idx, (img, true_label, pred_label) in enumerate(corr_class_samples[class_idx]):
			fig = visualize_gradcam(model, img, true_label, pred_label, device)
			plt.savefig(f'plots/{name}/gradcam_correct/gradcam_correct_c{class_idx}_{idx}_{name}.png', dpi=300, bbox_inches='tight')
			plt.show()

In [ ]:
print("정답 예측 샘플 (각 클래스 별 5개)")
incorr_class_samples = {i: [] for i in range(10)}
max_samples = 5
os.makedirs(f'plots/{name}/gradcam_incorrect', exist_ok=True)

for img, true_label, pred_label in incorrect_samples:
	curr = incorr_class_samples[true_label]
	if len(curr) < max_samples:
		curr.append((img, true_label, pred_label))

for class_idx in range(10):
	if incorr_class_samples[class_idx]:
		print(f"{class_idx} grad cam")
		for idx, (img, true_label, pred_label) in enumerate(incorr_class_samples[class_idx]):
			fig = visualize_gradcam(model, img, true_label, pred_label, device)
			plt.savefig(f'plots/{name}/gradcam_incorrect/gradcam_incorrect_c{class_idx}_{idx}_{name}.png', dpi=300, bbox_inches='tight')
			plt.show()

In [ ]:
name = f'resnet_{version}'

In [ ]:
# Feature Map 추출 및 시각화
class FeatureExtractor:
    """중간 레이어의 Feature Map 추출"""
    
    def __init__(self, model, layer_names):
        self.model = model
        self.layer_names = layer_names
        self.features = {}
        
        # Hook 등록
        for name, layer in model.named_modules():
            if name in layer_names:
                layer.register_forward_hook(self.save_feature(name))
    
    def save_feature(self, name):
        def hook(module, input, output):
            self.features[name] = output.detach()
        return hook
    
    def extract(self, x):
        self.features = {}
        _ = self.model(x)
        return self.features

def visualize_feature_maps(features, layer_name, max_channels=16):
    """Feature Map 시각화"""
    feature = features[layer_name][0]  # 첫 번째 배치
    num_channels = min(feature.shape[0], max_channels)
    
    # Grid 크기 계산
    grid_size = int(np.ceil(np.sqrt(num_channels)))
    
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
    axes = axes.flatten()
    
    for i in range(num_channels):
        fmap = feature[i].cpu().numpy()
        axes[i].imshow(fmap, cmap='viridis')
        axes[i].set_title(f'Ch {i}', fontsize=8)
        axes[i].axis('off')
    
    # 빈 subplot 숨기기
    for i in range(num_channels, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(f'Feature Maps: {layer_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    return fig

# Inception V4의 주요 레이어 이름 확인
print("\n" + "="*70)
print("🔍 모델 구조 탐색")
print("="*70)

for n, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        print(f"Conv Layer: {n}")

# 주요 레이어 선택 (예시)
target_layers = [
    'backbone.conv1.6',
    'backbone.layer1.0.conv3',
    'backbone.layer2.0.conv3',
    'backbone.layer4.0.conv3',
    'backbone.layer4.2.conv3'

]

# 샘플 이미지로 Feature Map 추출
sample_image, sample_label = next(iter(val_loader))
sample_image = sample_image[0:1].to(device)  # 첫 번째 이미지

extractor = FeatureExtractor(model, target_layers)
features = extractor.extract(sample_image)

# 각 레이어 시각화
for layer_name in target_layers:
    if layer_name in features:
        fig = visualize_feature_maps(features, layer_name, max_channels=16)
        plt.savefig(f'plots/{name}/feature_map_{layer_name.replace(".", "_")}.png', dpi=300)
        plt.show()

In [ ]:
# t-SNE/UMAP으로 임베딩 시각화
from sklearn.manifold import TSNE
# pip install umap-learn
# from umap import UMAP

def extract_embeddings(model, dataloader, device):
    """마지막 FC 레이어 이전의 임베딩 추출"""
    model.eval()
    embeddings = []
    labels = []
    
    # FC 레이어 이전 출력 추출을 위한 Hook
    features = []
    def hook(module, input, output):
        features.append(input[0].detach())
    

    handle = model.classifier.register_forward_hook(hook)
    
    with torch.no_grad():
        for images, lbls in tqdm(dataloader, desc='임베딩 추출'):
            images = images.to(device)
            _ = model(images)
            
            embeddings.append(features[-1].cpu().numpy())
            labels.extend(lbls.numpy())
            features.clear()
    
    handle.remove()
    
    embeddings = np.vstack(embeddings)
    labels = np.array(labels)
    
    return embeddings, labels

# 임베딩 추출
print("\n" + "="*70)
print("🧬 임베딩 추출 중...")
print("="*70)

embeddings, labels = extract_embeddings(model, val_loader, device)
print(f"임베딩 shape: {embeddings.shape}")

# t-SNE 시각화
print("\n📊 t-SNE 계산 중...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
embeddings_2d = tsne.fit_transform(embeddings)

plt.figure(figsize=(14, 10))
scatter = plt.scatter(
    embeddings_2d[:, 0], 
    embeddings_2d[:, 1], 
    c=labels, 
    cmap='tab10', 
    s=10, 
    alpha=0.6
)
plt.colorbar(scatter, label='Class', ticks=range(10))
plt.title('t-SNE Visualization of Learned Embeddings', fontsize=16, fontweight='bold')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(f'plots/{name}/tsne_embeddings.png', dpi=300)
plt.show()

# 클래스별로 색상 구분하여 Legend 추가
plt.figure(figsize=(16, 12))
for class_idx in range(10):
    mask = labels == class_idx
    plt.scatter(
        embeddings_2d[mask, 0],
        embeddings_2d[mask, 1],
        label=f'c{class_idx}',
        s=20,
        alpha=0.7
    )
plt.title('t-SNE Visualization by Class', fontsize=16, fontweight='bold')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(f'plots/{name}/tsne_embeddings_by_class.png', dpi=300)
plt.show()